This cell downloads the dataset from kaggle and imports important libraries for our code to run

In [1]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset
!pip install split-folders
import zipfile
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import splitfolders

Dataset URL: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset
License(s): other
 98% 772M/788M [00:03<00:00, 264MB/s]
100% 788M/788M [00:03<00:00, 265MB/s]


This cell extracts the zip file from the kaggle dataset and puts it in the "content" directory in a folder called "PetImages." Splitfolders creates a new folder called "output." Splitfolders takes the "PetImages" directory and splits it into 3 new subdirectories for training, validation, and testing based on the ratio parameter and puts these new subdirectories in the "output" folder.    

In [2]:
local_zip = '/content/microsoft-catsvsdogs-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

splitfolders.ratio('/content/PetImages/', output="output", seed=1337, ratio=(0.7, 0.1, 0.2))

Copying files: 25002 files [00:03, 6844.95 files/s]


This creates the file paths for our cat and dog training, validation, and testing directories

In [3]:
base_dir_train = '/content/output/train'
base_dir_val = '/content/output/val'
base_dir_test = '/content/output/test'

os.listdir(base_dir_train)
os.listdir(base_dir_val)
os.listdir(base_dir_test)

cat_train_dir = os.path.join(base_dir_train, "Cat")
dog_train_dir = os.path.join(base_dir_train, "Dog")

cat_val_dir = os.path.join(base_dir_val, "Cat")
dog_val_dir = os.path.join(base_dir_val, "Dog")

cat_test_dir = os.path.join(base_dir_test, "Cat")
dog_test_dir = os.path.join(base_dir_test, "Dog")

This function takes a folder path as an input and removes any images in the folder that cannot be loaded to prevent issues when training or testing the model

In [4]:
from PIL import Image

def check_images(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add other image extensions if needed
            filepath = os.path.join(folder_path, filename)
            try:
                img = Image.open(filepath)
            except Exception as e:
                print(f"Error loading image {filepath}: {e}")
                os.remove(filepath) # Remove the problematic file

We check all of our directories with the function above and remove any problematic images

In [5]:
check_images(cat_train_dir)
check_images(dog_train_dir)

check_images(cat_val_dir)
check_images(dog_val_dir)

check_images(cat_test_dir)
check_images(dog_test_dir)

Error loading image /content/output/train/Cat/666.jpg: cannot identify image file '/content/output/train/Cat/666.jpg'
Error loading image /content/output/train/Dog/11702.jpg: cannot identify image file '/content/output/train/Dog/11702.jpg'


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


We create our train and validation data generator here. It takes the training directory path, augments the data based on the parameters we set, and labels each image based on the name of the directory it came from. In our case, each image would be labeled a "Cat" or "Dog" based on the directory it came from.

In [6]:
img_width = 150
img_height = 150
batch_size = 40

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=40,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    base_dir_train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    base_dir_val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 17496 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


Here we create our convolutional neural network with 3 hidden layers and MaxPooling. We also have an output layer that uses a sigmoid function because we are doing a binary classification.

In [7]:
img_input = layers.Input(shape=(img_width, img_height, 3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)

model = Model(img_input, output)

Summarize the model

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496 

We now compile our model with the "adam" optimizer and train it using our train generator over 30 epochs. We also validate it using our validation generator.

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=30, validation_data=validation_generator)

Epoch 1/30
438/438 [==============================] - 124s 279ms/step - loss: 0.6698 - accuracy: 0.5854 - val_loss: 0.6075 - val_accuracy: 0.6536
Epoch 2/30
438/438 [==============================] - 121s 276ms/step - loss: 0.6027 - accuracy: 0.6714 - val_loss: 0.5745 - val_accuracy: 0.6976
Epoch 3/30
438/438 [==============================] - 120s 274ms/step - loss: 0.5655 - accuracy: 0.7047 - val_loss: 0.5161 - val_accuracy: 0.7416
Epoch 4/30
438/438 [==============================] - 121s 276ms/step - loss: 0.5331 - accuracy: 0.7311 - val_loss: 0.4802 - val_accuracy: 0.7820
Epoch 5/30
438/438 [==============================] - 122s 277ms/step - loss: 0.5183 - accuracy: 0.7429 - val_loss: 0.5071 - val_accuracy: 0.7456
Epoch 6/30
438/438 [==============================] - 121s 276ms/step - loss: 0.4955 - accuracy: 0.7546 - val_loss: 0.4543 - val_accuracy: 0.7956
Epoch 7/30
438/438 [==============================] - 121s 275ms/step - loss: 0.4912 - accuracy: 0.7613 - val_loss: 0.4651 -

We can see we achieved an accuracy of around 85.8%. To increase accuracy, we can increase the amount of training images used by changing the ratio parameter in splitfolders. However, this comes with the tradeoff of taking more time to train our model. For example, if we use 7,500 images to train, our model will take about half the time to train, and we will only lose about 3% accuracy. Our validation accuracy is 88.1%, which suggests that we are not overfitting.

Below we create a function to process images before we try to classify them with our model

In [10]:
from keras.preprocessing import image

def preprocess_image(img_path):
  img = image.load_img(img_path, target_size=(img_width, img_height))  # Load image and resize
  img = image.img_to_array(img)  # Convert to numpy array
  img = np.expand_dims(img, axis=0)  # Add batch dimension
  img /= 255.0  # Normalize pixel values
  return img

Here we try classifying one our pictures from the "Cat" directory and we see that our model correctly identifies it as a Cat.

In [11]:
image_path = '/content/PetImages/Cat/10040.jpg'
preprocessed_image = preprocess_image(image_path)
prediction = model.predict(preprocessed_image)

if prediction[0][0] < 0.5:
  print('Predicted class: Cat')
else:
  print('Predicted class: Dog')

1/1 [==============================] - 0s 115ms/step
Predicted class: Cat
